In [120]:
import numpy as np
from itertools import product
import pandas as pd
import os
import foraging_toolkit as ft
import environments
import simulation
import simulation_old 
import success_metrics
import imp
imp.reload(ft)
imp.reload(simulation)
imp.reload(simulation_old)
imp.reload(environments)
imp.reload(success_metrics)


<module 'success_metrics' from 'c:\\Users\\admin\\Dropbox\\Code\\Basis-code\\new-collaborative-intelligence\\mx_refactored\\success_metrics.py'>

Independent variables in the forward model. Specify the parameter values for all the cases you are interested in. 

In [121]:
# Parameters of interest in our communication model 
c_trust = np.array([0, 0.5, 1])
sight_radius = [5]
reward_patch_dim = [1, 4]

# List of all possible combination of parameter values 
param_list = [i for i in product(c_trust, sight_radius, reward_patch_dim)]

print(len(param_list))
print('param_list', param_list)




6
param_list [(0.0, 5, 1), (0.0, 5, 4), (0.5, 5, 1), (0.5, 5, 4), (1.0, 5, 1), (1.0, 5, 4)]


Generate a dataframe containing all possible combinations of the parameter values specified above.

In [122]:
#each row of both dataframes corresponds to one experiment.  
#columns are parameters 
metadataDF = pd.DataFrame(param_list)

# success_metricsDF = pd.DataFrame(results)
# allresultsDF = pd.concat([metadataDF, success_metricsDF], axis='columns') # concatenate metadata and results 
metadataDF.columns = ["c_trust", "sight_radius", "reward_patch_dim"] 
metadataDF['sim index'] = np.arange(len(metadataDF)).astype(int)
metadataDF.head()
display(metadataDF)

,c_trust,sight_radius,reward_patch_dim,sim index
0,0.0,5,1,0
1,0.0,5,4,1
2,0.5,5,1,2
3,0.5,5,4,3
4,1.0,5,1,4
5,1.0,5,4,5


In [123]:
# select a specific row of the data frame
ind = 1
row = metadataDF.iloc[[ind]]
print(row)
print(row['c_trust'].iloc[0])
print(row['sight_radius'].iloc[0])
print(row['reward_patch_dim'].iloc[0])

   c_trust  sight_radius  reward_patch_dim  sim index
1      0.0             5                 4          1
0.0
5
4


In [124]:
test_arr = np.zeros((5))
print(test_arr.shape)

(5,)


*Part 2*
Test run one of the simulations from metadata (one row)

In [139]:
N_frames = 20
N_agents = 2
N_total_food_units = 16
edge_size = 30

nsims = len(metadataDF)
N_runs = 3
nsims = 2 # TEST

# data frame to store all success values across all simulations and runs of each simulation
success_measures_allDF = pd.DataFrame() 
success_sims_list = []

home_dir = "C:\\Basis_data\\" # 

# save metadata to home directory
metadataDF.to_csv(home_dir + '\metadataDF' + '.csv')

for si in range(nsims):
    
    # 1. pull out parameters from row si in the metadata
    df_row = metadataDF.iloc[[si]]
    c_trust = df_row['c_trust'].iloc[0]
    sight_radius = df_row['sight_radius'].iloc[0]
    reward_patch_dim = df_row['reward_patch_dim'].iloc[0].astype(int)

    # print(df_row)
    # print(reward_patch_dim)

    mean_times_to_first_reward = np.zeros((N_runs))
    num_birds_failed = np.zeros((N_runs))

    # Do multiple runs of the simulation and store the results in a results dataframe
    for ri in range(N_runs):
        # 2. Feed the parameters through the run() function once (later we'll do this many times)
        env = environments.Environment(edge_size=edge_size, N_total_food_units=N_total_food_units, patch_dim=reward_patch_dim)
        env.add_food_patches() #food_statistics_type="drop_food_once")
        # sim = simulation_old.Simulation(env, N_agents=N_agents, N_frames=N_frames)
        sim = simulation.SimulateCommunicators(env, N_frames, N_agents, c_trust=c_trust, sight_radius=sight_radius)
        sim.run()
        # print(sim.all_birdsDF)
        # # print(sim.all_rewardsDF) 
        # # by this point we should have birdlocsDF and rewardlocsDF 
        

        # # # 3. Compute success measures 
        
        time_to_first_allbirds = np.zeros(N_agents)
        for bird_id in range(1, N_agents + 1):
            singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id]  
            time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)
            
        # these are the success measures
        mean_times_to_first_reward[ri] = np.mean(time_to_first_allbirds) # take the average across birds
        num_birds_failed[ri] = np.sum(time_to_first_allbirds < 0) # number of birds that failed to reach food
        # print(num_birds_failed)
        #

        # SAVE THE DATA 
        # make a new folder with 
        sim_folder = 'sim' + str(si) + '_run' + str(ri)
        sim_dir = home_dir + sim_folder
        if not os.path.isdir(sim_dir):
            os.makedirs(sim_dir)
            print("Directory %s was created." %sim_folder)

        birdfile = '\sim' + str(si) + '_run' + str(ri) + '_birdlocsDF'
        rewardfile = '\sim' + str(si) + '_run' + str(ri) + '_rewardlocsDF'
        sim.all_birdsDF.to_csv(sim_dir + birdfile + ".csv")
        sim.all_rewardsDF.to_csv(sim_dir + rewardfile + ".csv")

       

    #4. Combine the metadata and the success measures into the results dataframe 
        # create a new row in the results data frame, then concatenate 
    success_measures_onesim_DF = pd.DataFrame(
        {
            "run index": np.arange(N_runs),
            "time to first food": mean_times_to_first_reward,
            "num birds failed": num_birds_failed,
        }
    )

    success_sims_list.append(success_measures_onesim_DF)

success_measures_allDF = pd.concat(success_sims_list)

    # 5. Save the raw data into a separate folder.


print(len(success_sims_list))
print(len(mean_times_to_first_reward))
print(len(success_measures_onesim_DF))
print(display(success_measures_onesim_DF))
print(display(success_measures_allDF))

# Concatenate metadataDF and success_measures_allDF into a new dataframe called resultsDF

    
# Save the results dataframe into the same folder as the metadata


2
3
3


,run index,time to first food,num birds failed
0,0,8.0,0.0
1,1,-1.0,2.0
2,2,1.5,1.0


None


,run index,time to first food,num birds failed
0,0,3.0,0.0
1,1,1.5,0.0
2,2,1.0,0.0
0,0,8.0,0.0
1,1,-1.0,2.0
2,2,1.5,1.0


None


In [136]:
sim_folder = 'sim' + str(si) + '_run' + str(ri) 
sim_dir = home_dir + sim_folder


birdfile = '\sim' + str(si) + '_run' + str(ri) + '_birdlocsDF'


print(sim_dir + birdfile)

C:\Basis_data\sim1_run2\sim1_run2_birdlocsDF


In [126]:
# Concatenate metadataDF and success_measures_allDF into a new dataframe called resultsDF

# First, duplicate each row (sim) of metadataDF by the number of runs
newDF = pd.DataFrame(np.repeat(metadataDF.values, N_runs, axis=0))
newDF.columns = metadataDF.columns
print(newDF)




    c_trust  sight_radius  reward_patch_dim  sim index
0       0.0           5.0               1.0        0.0
1       0.0           5.0               1.0        0.0
2       0.0           5.0               1.0        0.0
3       0.0           5.0               4.0        1.0
4       0.0           5.0               4.0        1.0
5       0.0           5.0               4.0        1.0
6       0.5           5.0               1.0        2.0
7       0.5           5.0               1.0        2.0
8       0.5           5.0               1.0        2.0
9       0.5           5.0               4.0        3.0
10      0.5           5.0               4.0        3.0
11      0.5           5.0               4.0        3.0
12      1.0           5.0               1.0        4.0
13      1.0           5.0               1.0        4.0
14      1.0           5.0               1.0        4.0
15      1.0           5.0               4.0        5.0
16      1.0           5.0               4.0        5.0
17      1.

In [127]:
# change the indexing of the rows of this dataframe so we can concatenate with metadataDF
successDF = success_measures_allDF.reset_index(drop=True)
display(successDF)

,run index,time to first food,num birds failed
0,0,5.0,0.0
1,1,1.5,0.0
2,2,1.5,0.0
3,0,-1.0,2.0
4,1,0.5,1.0
5,2,4.0,1.0


In [128]:
# Concatenate metadataDF and 
allresultsDF = pd.concat([newDF, successDF], axis=1)
display(allresultsDF)

,c_trust,sight_radius,reward_patch_dim,sim index,run index,time to first food,num birds failed
0,0.0,5.0,1.0,0.0,0.0,5.0,0.0
1,0.0,5.0,1.0,0.0,1.0,1.5,0.0
2,0.0,5.0,1.0,0.0,2.0,1.5,0.0
3,0.0,5.0,4.0,1.0,0.0,-1.0,2.0
4,0.0,5.0,4.0,1.0,1.0,0.5,1.0
5,0.0,5.0,4.0,1.0,2.0,4.0,1.0
6,0.5,5.0,1.0,2.0,NaN,NaN,NaN
7,0.5,5.0,1.0,2.0,NaN,NaN,NaN
8,0.5,5.0,1.0,2.0,NaN,NaN,NaN
9,0.5,5.0,4.0,3.0,NaN,NaN,NaN


*Save data in a folder*


In [129]:
# test saving functions
data_dir = "C:\\Basis_data\\"
file_name = "testfile"
date_folder_name = 
metadataDF.to_csv(data_dir + file_name + ".csv")

SyntaxError: invalid syntax (2446219485.py, line 4)

Test one of the simulations by animating it.

In [ ]:
communicators = ft.object_from_data(sim.all_birdsDF, sim.all_rewardsDF)
ft.animate_birds(communicators, plot_rewards=True,
                  width = 600, height = 600, point_size = 10)

In [ ]:
print(sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == 1])

       x     y  time  bird           type
0   25.0  26.0     1     1  communicators
1   26.0  24.0     2     1  communicators
2   24.0  26.0     3     1  communicators
3   27.0  26.0     4     1  communicators
4   25.0  28.0     5     1  communicators
5   26.0  28.0     6     1  communicators
6   28.0  28.0     7     1  communicators
7   28.0  25.0     8     1  communicators
8   29.0  26.0     9     1  communicators
9   26.0  26.0    10     1  communicators
10  26.0  25.0    11     1  communicators
11  28.0  23.0    12     1  communicators
12  28.0  26.0    13     1  communicators
13  29.0  26.0    14     1  communicators
14  28.0  25.0    15     1  communicators
15  28.0  22.0    16     1  communicators
16  29.0  22.0    17     1  communicators
17  27.0  21.0    18     1  communicators
18  27.0  20.0    19     1  communicators
19   0.0   0.0    20     1  communicators


In [ ]:
# print(sim.all_rewardsDF[]) 
print(sim.all_rewardsDF.loc[sim.all_rewardsDF['time'] == 4])


     x   y  time
0   20  14     4
1   21  14     4
2   22  14     4
3   23  14     4
4   20  15     4
5   21  15     4
6   22  15     4
7   23  15     4
8   20  16     4
9   21  16     4
10  22  16     4
11  23  16     4
12  20  17     4
13  21  17     4
14  22  17     4
15  23  17     4


In [ ]:

time_to_first_allbirds = np.zeros(N_agents)
# print(time_to_first_allbirds.shape)

for bird_id in range(1, N_agents + 1):
    # bird_id = 2
    singlebirdDF = sim.all_birdsDF.loc[sim.all_birdsDF['bird'] == bird_id] 
    display(singlebirdDF)
    time_to_first_allbirds[bird_id - 1] = success_metrics.compute_time_to_first_reward(singlebirdDF, sim.all_rewardsDF, N_frames)

    print(time_to_first_allbirds)

num_birds_failed = np.sum(time_to_first_allbirds < 0)
print('Num birds failed to find food = ' + str(num_birds_failed))


,x,y,time,bird,type
0,25.0,26.0,1,1,communicators
1,26.0,24.0,2,1,communicators
2,24.0,26.0,3,1,communicators
3,27.0,26.0,4,1,communicators
4,25.0,28.0,5,1,communicators
5,26.0,28.0,6,1,communicators
6,28.0,28.0,7,1,communicators
7,28.0,25.0,8,1,communicators
8,29.0,26.0,9,1,communicators
9,26.0,26.0,10,1,communicators


[-1.  0.]


,x,y,time,bird,type
0,18.0,8.0,1,2,communicators
1,19.0,6.0,2,2,communicators
2,21.0,8.0,3,2,communicators
3,18.0,8.0,4,2,communicators
4,17.0,10.0,5,2,communicators
5,16.0,8.0,6,2,communicators
6,13.0,8.0,7,2,communicators
7,12.0,8.0,8,2,communicators
8,10.0,8.0,9,2,communicators
9,12.0,10.0,10,2,communicators


[-1. -1.]
Num birds failed to find food = 2
